
# Exercise 11: Visual Odometry

Solutions for the week 11 exercises. We follow the lecture terminology on calibrated two-view geometry and visual odometry.



## Exercise 11.1

Detect SIFT keypoints and descriptors in the first three images and match them between consecutive frames. We limit the number of SIFT features to 2000 and store keypoints as 2D numpy arrays.


In [ ]:

import numpy as np
import cv2
import matplotlib.pyplot as plt
from pathlib import Path

data_dir = Path('exercises/ex11_data')
K = np.loadtxt(data_dir / 'K.txt')

# Load the first three images
im0 = cv2.imread(str(data_dir / '000001.png'), cv2.IMREAD_GRAYSCALE)
im1 = cv2.imread(str(data_dir / '000002.png'), cv2.IMREAD_GRAYSCALE)
im2 = cv2.imread(str(data_dir / '000003.png'), cv2.IMREAD_GRAYSCALE)

# Detect SIFT features (at most 2000)
sift = cv2.SIFT_create(nfeatures=2000)
kp0, des0 = sift.detectAndCompute(im0, None)
kp1, des1 = sift.detectAndCompute(im1, None)
kp2, des2 = sift.detectAndCompute(im2, None)

# Convert keypoints to arrays of 2D points
kp0 = np.array([k.pt for k in kp0])
kp1 = np.array([k.pt for k in kp1])
kp2 = np.array([k.pt for k in kp2])

# Match SIFT features between consecutive frames
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
matches01 = bf.match(des0, des1)
matches12 = bf.match(des1, des2)

# Convert matches to arrays of indices
matches01 = np.array([(m.queryIdx, m.trainIdx) for m in matches01])
matches12 = np.array([(m.queryIdx, m.trainIdx) for m in matches12])



## Exercise 11.2

Estimate the essential matrix between the first two images using RANSAC with `cv2.findEssentialMat`. Decompose it with `cv2.recoverPose` and retain only matches that are inliers and lie in front of both cameras.


In [ ]:

# Estimate essential matrix and recover relative pose
E, mask_E = cv2.findEssentialMat(kp0[matches01[:,0]], kp1[matches01[:,1]], K, method=cv2.RANSAC, prob=0.999, threshold=1.0)
_, R1, t1, mask_pose = cv2.recoverPose(E, kp0[matches01[:,0]], kp1[matches01[:,1]], K, mask=mask_E)

# Keep matches that are inliers and in front of both cameras
mask = mask_E.ravel().astype(bool) & mask_pose.ravel().astype(bool)
matches01 = matches01[mask]



## Exercise 11.3

Find feature tracks across the first three frames so that `points0[i]`, `points1[i]`, and `points2[i]` denote the same scene point.


In [ ]:

_, idx01, idx12 = np.intersect1d(matches01[:,1], matches12[:,0], return_indices=True)
points0 = kp0[matches01[idx01][:,0]]
points1 = kp1[matches01[idx01][:,1]]
points2 = kp2[matches12[idx12][:,1]]



## Exercise 11.4

Triangulate 3‑D points from the first two images and estimate the pose of the third image with `cv2.solvePnPRansac`. Visualize the inlier 3‑D points together with the camera centres (the camera centre is given by −R^T t).


In [ ]:

# Projection matrices for views 0 and 1
P0 = K @ np.hstack([np.eye(3), np.zeros((3,1))])
P1 = K @ np.hstack([R1, t1])

# Triangulate homogeneous points and convert to Euclidean coordinates
hom_Q = cv2.triangulatePoints(P0, P1, points0.T, points1.T)
Q = (hom_Q[:3] / hom_Q[3]).T

# Estimate pose of third image with PnP RANSAC
retval, rvec2, tvec2, inliers = cv2.solvePnPRansac(Q, points2, K, distCoeffs=np.zeros(5))
R2, _ = cv2.Rodrigues(rvec2)

# Helper for camera centre
camera_center = lambda R, t: (-R.T @ t).ravel()

# Visualize
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(*Q[inliers.flatten()].T, s=5)
ax.scatter(*camera_center(np.eye(3), np.zeros((3,1))), c='r')
ax.scatter(*camera_center(R1, t1), c='g')
ax.scatter(*camera_center(R2, tvec2), c='b')
ax.set_xlabel('X'); ax.set_ylabel('Y'); ax.set_zlabel('Z');



## Exercise 11.5

Generalize the visual odometry pipeline to the complete sequence by iterating over all images. We maintain lists of rotations and translations (`Rs`, `ts`) and triangulate new points for each step.


In [ ]:

# Iterate through all images in the folder
image_files = sorted(data_dir.glob('*.png'))
Rs = [np.eye(3), R1]
ts = [np.zeros((3,1)), t1]
all_points = []

for i in range(2, len(image_files)):
    img_prev = cv2.imread(str(image_files[i-1]), cv2.IMREAD_GRAYSCALE)
    img_curr = cv2.imread(str(image_files[i]), cv2.IMREAD_GRAYSCALE)
    kp_prev, des_prev = sift.detectAndCompute(img_prev, None)
    kp_curr, des_curr = sift.detectAndCompute(img_curr, None)
    kp_prev = np.array([k.pt for k in kp_prev])
    kp_curr = np.array([k.pt for k in kp_curr])
    matches = bf.match(des_prev, des_curr)
    matches = np.array([(m.queryIdx, m.trainIdx) for m in matches])
    E, mask_E = cv2.findEssentialMat(kp_prev[matches[:,0]], kp_curr[matches[:,1]], K, method=cv2.RANSAC, prob=0.999, threshold=1.0)
    _, R, t, mask_pose = cv2.recoverPose(E, kp_prev[matches[:,0]], kp_curr[matches[:,1]], K, mask=mask_E)
    mask = mask_E.ravel().astype(bool) & mask_pose.ravel().astype(bool)
    matches = matches[mask]
    pts_prev = kp_prev[matches[:,0]]
    pts_curr = kp_curr[matches[:,1]]
    P_prev = K @ np.hstack([Rs[-1], ts[-1]])
    P_curr = K @ np.hstack([Rs[-1] @ R, ts[-1] + t])
    hom = cv2.triangulatePoints(P_prev, P_curr, pts_prev.T, pts_curr.T)
    Q_new = (hom[:3] / hom[3]).T
    all_points.append(Q_new)
    Rs.append(Rs[-1] @ R)
    ts.append(ts[-1] + t)

# Stack all 3-D points
all_points = np.vstack(all_points)



## Optional extensions

- **Exercise 11.6:** Triangulate each feature using all frames where it was observed.
- **Exercise 11.7:** Capture your own calibrated image sequence and apply the pipeline.
- **Exercise 11.8:** Perform bundle adjustment over all poses and 3‑D points for refinement.
- **Exercise 11.9:** Evaluate the method on the KITTI sequence 09 with provided ground truth.
